In [1]:
import pandas as pd
import mysql.connector

In [63]:
host = "feenix-mariadb.swin.edu.au"
user = "s104489467"
password = "101100"
database = "s104489467_db"
connection = mysql.connector.connect(host=host,user=user,password=password,database=database)
cursor = connection.cursor()

In [3]:
sql_query = "select * from public_sentiment"

In [37]:
df = pd.read_sql(sql_query, connection)

C:\Users\HP Envy\AppData\Local\Temp\ipykernel_2844\1690567701.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql_query, connection)


In [38]:
df['key_word'] = df['key_word'].str.strip() \
    .str.replace('-', '', regex=False) 

In [39]:
df['key_word'] = df['key_word'].str.replace(r'\\n', ', ', regex=True)

In [40]:
df['key_word'] = df['key_word'].str.replace(r'\*', ', ', regex=True) 
df['key_word'] = df['key_word'].str.replace(r'\d+', ', ', regex=True) 

In [44]:
df['key_word'] =df['key_word'].str.replace(r', , ',', ')

In [46]:
df['key_word'] =df['key_word'].str.replace(r' , ',', ')

In [48]:
df['key_word'] = df['key_word'].str.split(',').apply(lambda x: ', '.join(sorted(set(x), key=x.index)))

In [50]:
df['key_word'] =df['key_word'].str.replace(r'.','')

In [52]:
df['key_word'] =df['key_word'].str.replace(r':','')

In [54]:
df['key_word'] = df['key_word'].str.lstrip("', ")

In [56]:
df['key_word'] =df['key_word'].str.replace(r'(Note Notwithstanding and excessively is used here)','')

In [73]:
import pandas as pd
import nltk

# Ensure necessary NLTK data is downloaded
nltk.download('punkt')
def extract_capitalized_tokens(sentence):
    # Tokenize the sentence
    tokens = nltk.word_tokenize(sentence)
    # Filter tokens that start with a capitalized first letter
    capitalized_tokens = [token for token in tokens if token[0].isupper()]
    return ', '.join(capitalized_tokens)
df['capitalized_tokens'] = df['key_word'].apply(extract_capitalized_tokens)

[nltk_data] Downloading package punkt to C:\Users\HP
[nltk_data]     Envy\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [92]:
df['capitalized_tokens'] = df['capitalized_tokens'].str.replace(r'Keyword','')

In [83]:
df['key_word'] = df['key_word'].str.lstrip(", ")

In [85]:
df['capitalized_tokens'] = df['capitalized_tokens'].str.replace(r'None, ','')


In [87]:
df['capitalized_tokens'] = df['capitalized_tokens'].str.replace(r' As, I, , ','')


In [88]:
df['capitalized_tokens'] = df['capitalized_tokens'].str.replace(r'Here, We, , ','')


In [89]:
df['capitalized_tokens'] = df['capitalized_tokens'].str.replace(r', Tax, A, , Technology, The, LNP\\','')


In [90]:
df['capitalized_tokens'] = df['capitalized_tokens'].str.replace(r' Brain, Drain,','')


In [91]:
df['capitalized_tokens'] = df['capitalized_tokens'].str.replace(r' No, Economy, Yes, Tax, No, Investment, Yes, Technology, Yes, Environment, Yes, Cost, Yes','')


In [95]:
df.drop(['key_word'], axis=1, inplace=True)

In [96]:
df

,id,country,energy_source,comment,sentiment_score,capitalized_tokens
0,61,Australia,FMAA,https://12ft.io/https://www.smh.com.au/politic...,1.5632,"Policy, Jobs"
1,62,Australia,FMAA,The Coalition has attacked the production cred...,1.8852,"Tax, Investment, Environment, Cost"
2,63,Australia,FMAA,Production credits are better than handouts - ...,4.5928,"Jobs, Cost"
3,64,Australia,FMAA,"Roll the dice with our labour rates, our envir...",3.0,"Market, Risk, Investment, Strategy, Cost, Mana..."
4,65,Australia,FMAA,"Aiming for more economic complexity is good, b...",3.807,"Policy, Economy, Cost, Investment, Technology"
...,...,...,...,...,...,...
3771,3832,Australia,Other,Despite calls for no new #coal plants in order...,1,"Tax, Investment, Environment"
3772,3833,Australia,Other,Curious crossover: a political funding group i...,2,"Policy, Tax"
3773,3834,Australia,Other,Momentum is building for stronger action on cl...,4,"Environment, Climate, Change"
3774,3835,Australia,Nuclear Energy,Australia politics live: anti-trans protests ‘...,1,"Policy, Jobs, Economy, Note, There"


In [69]:
for i in range(len(df)):
    country = df['country'][i]
    energy_source = df['energy_source'][i]
    comment = df['comment'][i]
    key_word = df['capitalized_tokens'][i]
    sentiment_score = df['sentiment_score'][i]
    query="Insert into public_sentiment (country, energy_source, comment, key_word, sentiment_score) values (%s, %s, %s, %s, %s)"
    cursor.execute(query, (country, energy_source, comment, key_word, sentiment_score))
    connection.commit()
cursor.close()
connection.close()

In [93]:
a = df['capitalized_tokens'].to_list()
a

['Policy, Jobs',
 'Tax, Investment, Environment, Cost',
 'Jobs, Cost',
 'Market, Risk, Investment, Strategy, Cost, Management, Policy, Infrastructure',
 'Policy, Economy, Cost, Investment, Technology',
 'Policy, Jobs, Cost',
 'Policy, Policy, Renewables',
 'Policy, Investment, Technology, Environment',
 'Policy, Investment, Technology, Jobs, Economy, Tax',
 'Jobs, Investment',
 'Policy, Investment, Tax, Infrastructure, Technology',
 'Policy, Jobs, Investment, Technology, Environment',
 'Jobs',
 'Policy, Cost, Investment',
 'Policy, Jobs, Technology',
 'Policy, Tax, Investment, Technology, Policy, Jobs, Environment',
 'Policy, Economy, Jobs, Tax, Investment',
 'Policy, Economy, Investment, Environment, Tax, Infrastructure',
 ', Policy, Investment',
 'Policy, Jobs, Economy, Tax, Investment, Technology, Environment, Cost, Infrastructure',
 'Policy, Investment, Economy, Jobs, Productivity, Cost, Teriff, Environment, Industry',
 'Policy, Jobs',
 'Jobs, Economy',
 'Tax, Policy, Costello',
 '